## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [23]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [24]:
load_dotenv(override=True)
openai = OpenAI()

In [25]:
reader = PdfReader("me/KinjalShah_Resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [26]:
print(resume)

KINJAL SHAH
510-468-2232| ShahKinjal@gmail.com| www.linkedin.com/in/ShahKinjal
SUMMARY
Experienced engineering leader with a strong background in software engineering and product development, skilled in leading 
cross-functional teams to drive innovation and exceed goals. Successfully scaled engineering teams and developed a fault-tolerant 
roadside assistance platform processing over 1M transactions weekly. Aims to leverage technical expertise and leadership to foster growth 
and operational excellence as an Engineering Director/VP.
CORE COMPETENCIES
• Cloud: AWS , GCP , Cloud Migration, Multi-Tenant Cloud System, Redshift Serveless
• Technology: Microservices, GraphOL , ReST Web Services, Blockchain, Elastic Search, Kafka, Java, Spring Boot, Split.io, Log4j, 
Linux & Apache, Couchbase, Apigee
• Security: Security (PCI, GDPR , ISO27001, CCPA) Compliance, Secure Coding Standards, Hashicorp vault, Data Security, Identity 
and Access Management (IAM)
• Design: Cloud & Cloud Native Platfo

In [29]:
profile = PdfReader("me/linkedin.pdf")
profile = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        profile += text

In [30]:
print(profile)

KINJAL SHAH
510-468-2232| ShahKinjal@gmail.com| www.linkedin.com/in/ShahKinjal
SUMMARY
Experienced engineering leader with a strong background in software engineering and product development, skilled in leading 
cross-functional teams to drive innovation and exceed goals. Successfully scaled engineering teams and developed a fault-tolerant 
roadside assistance platform processing over 1M transactions weekly. Aims to leverage technical expertise and leadership to foster growth 
and operational excellence as an Engineering Director/VP.
CORE COMPETENCIES
• Cloud: AWS , GCP , Cloud Migration, Multi-Tenant Cloud System, Redshift Serveless
• Technology: Microservices, GraphOL , ReST Web Services, Blockchain, Elastic Search, Kafka, Java, Spring Boot, Split.io, Log4j, 
Linux & Apache, Couchbase, Apigee
• Security: Security (PCI, GDPR , ISO27001, CCPA) Compliance, Secure Coding Standards, Hashicorp vault, Data Security, Identity 
and Access Management (IAM)
• Design: Cloud & Cloud Native Platfo

In [5]:
with open("me/ks_summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [31]:
name = "Kinjal Shah"

In [32]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background,  LinkedIn profile and resumewhich you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{profile}\n\n## Resume:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [33]:
system_prompt

"You are acting as Kinjal Shah. You are answering questions on Kinjal Shah's website, particularly questions related to Kinjal Shah's career, background, skills and experience. Your responsibility is to represent Kinjal Shah for interactions on the website as faithfully as possible. You are given a summary of Kinjal Shah's background,  LinkedIn profile and resumewhich you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Kinjal Shah. I'm a software engineer and a leader. I'm originally from Vadodara, India, but I moved to CA in 1996. I am currelty learing AI and agentic ai engineering. \nI a vegetarian and I love savory food. Indian food. I recenlty gottten in to Healty eating and living and working out. I work out 5 days a week. I have recenlty lost 40 lbs of weight and gained xonsierable amount of muscle mass. \n\n## LinkedIn Pr

In [34]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [35]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [37]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [38]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and Resume details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{profile}\n\n## Resume:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [39]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [40]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [41]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

"I currently do not hold any patents. My focus has predominantly been on software engineering, product development, and leading teams to innovate solutions in the tech industry. However, I'm always open to exploring new ideas and innovations in my field. If you have a particular project in mind, I'd be happy to discuss!"

In [ ]:
evaluate(reply, "do you hold a patent?", messages[:1])

In [42]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [43]:
def chat(message, history):
    #if "patent" in message:
    system = system_prompt 
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [44]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
